# Use examples of [premise](https://github.com/romainsacchi/premise) with user-generated scenarios

Author: [romainsacchi](https://github.com/romainsacchi)

This notebook shows examples on how to use `premise` to adapt the life cycle inventory database [ecoinvent](https://www.ecoinvent.org/) for prospective environmental impact assessment, using **user-generated scenarios**.

User-generated scenario are scenarios built by the premise users community. They can be used on their own
or together with a global IAM scenario.
Public community scenarios are listed under the repository [Premise community scenario](https://github.com/premise-community-scenarios).

User-generated scenarios are "packaged" into [data packages](https://specs.frictionlessdata.io/data-package/).

Data packages ensure that the data used comes with all necessary metadata, resources (scenario data, inventories, etc.), and that the data is formatted in a correct way.

To fetch a data packge, you can use the `datapackage` library.
For exmaple, let's fetch the user-generated scenario about [Switzerland's future energy supply](https://github.com/premise-community-scenarios/energy-perspective-2050-switzerland).

In [59]:
from premise import *

clear_cache()

Cache folder cleared!


In [123]:
from datapackage import Package

In [124]:
# URL pointing to the raw datapackage.json file of the scenario (on github or computer)
#url = r"https://raw.githubusercontent.com/alideoro/Team_PV/main/energy-perspective-2050-switzerland-main2/datapackage.json"
url = r"https://raw.githubusercontent.com/alideoro/Team_PV/main/energy-perspective-2050-switzerland-main/datapackage.json"

switzerland_2050 = Package(url)

The datapackage has 3 resources: scenario data, inventories and a configuration file.

In [73]:
for resource in switzerland_2050.resources:
    print(resource.name)

scenario_data
inventories
config


or

In [74]:
switzerland_2050.resource_names

['scenario_data', 'inventories', 'config']

And you can directly read them (or look the resources up directly from the repo):

In [75]:
import pandas as pd

In [125]:
#data.raw_read()

data2 = switzerland_2050.get_resource("scenario_data").read()
df=pd.DataFrame(data2)

In [126]:
df.loc[:, 4].unique()

array(['Production|Electricity|Run-of-river hydro',
       'Production|Electricity|Reservoir',
       'Production|Electricity|Nuclear|Boiling water reactor',
       'Production|Electricity|Nuclear|Pressure water reactor',
       'Production|Electricity|Conventional|Waste-to-Energy',
       'Production|Electricity|Conventional|Other',
       'Production|Electricity|Renewable|Photovoltaic_commercial',
       'Production|Electricity|Renewable|Photovoltaic2',
       'deple_pv_monoSI',
       'Production|Electricity|Renewable|Photovoltaic_residential',
       'Production|Electricity|Renewable|Wind turbines|Onshore',
       'Production|Electricity|Renewable|Geothermal',
       'Production|Electricity|Renewable|Biomass',
       'Production|Electricity|Renewable|Biogas',
       'Production|Electricity|Medium to high',
       'Production|Electricity|Low to medium',
       'Production|Electricity|Import',
       'Transmission losses|Electricity|Medium to high',
       'Distribution losses|Electr

The datapackage can also be locally stored (i.e., on your computer), in which case you only
need to provide the filepath to the `datapackage.json` file.

Once the datapackage is loaded, you just need to pass it (in a list) to the `NewDatabase` instance of `premise`.

In [127]:
from premise import *
import bw2data
bw2data.projects.set_current("ei38-teaching")
bw2data.databases

Databases dictionary with 7 object(s):
	TeamPV
	TeamPV2
	TeamPV3
	TeamPV4
	TeamPV5
	biosphere3
	ei 3.8 cutoff

In [128]:
scenarios = [
    {"model": "image", "pathway":"SSP2-Base", "year": 2030},
    #{"model": "image", "pathway":"SSP2-Base", "year": 2040},             # multiple scenarios can be specified
    #{"model": "image", "pathway":"SSP2-Base", "year": 2050},
]

ndb = NewDatabase(
        scenarios = scenarios,        
        source_db="ei 3.8 cutoff",
        source_version="3.8",
        key='tUePmX_S5B8ieZkkM7WUU2CnO8SmShwmAeWK9x2rTFo=',
        external_scenarios=[
            switzerland_2050,
        ]
)


premise v.(1, 3, 6)
+------------------------------------------------------------------+
| Warning                                                          |
+------------------------------------------------------------------+
| Because some of the scenarios can yield LCI databases            |
| containing net negative emission technologies (NET),             |
| it is advised to account for biogenic CO2 flows when calculating |
| Global Warming potential indicators.                             |
| `premise_gwp` provides characterization factors for such flows.  |
| It also provides factors for hydrogen emissions to air.          |
|                                                                  |
| Within your bw2 project:                                         |
| from premise_gwp import add_premise_gwp                          |
| add_premise_gwp()                                                |
+------------------------------------------------------------------+
+-------------

To integrate the projections of the user-generated scenario, call `update_external_scenario()`.

In [129]:
ndb.update_external_scenario()


//////////////// IMPORTING USER-DEFINED INVENTORIES ////////////////
Extracted 1 worksheets in 0.00 seconds
Create custom markets.
Log file of exchanges saved under /opt/tljh/user/envs/bw/lib/python3.10/site-packages/premise/data/logs.


You can combine the user-generated scenario with any, all or none of the projections for the IAM model.
For example, here with the electricity projections of the IMAGE SSP2-Base scenario:

In [130]:
temp_db = ndb.scenarios[0]["database"]

for act in temp_db:
    if "EP2050" in act["name"] and "market for electricity" in act["name"]:
        for exc in act["exchanges"]:
            if exc["type"] == "technosphere":
                print(act["name"], act["location"], exc["name"], exc["amount"], exc["location"])

market for electricity, from neighboring countries, high voltage, EP2050 CH electricity production, hydro, reservoir, alpine region 0.13059964701855462 CH
market for electricity, from neighboring countries, high voltage, EP2050 CH electricity production, nuclear, boiling water reactor 0.22440993513818197 CH
market for electricity, from neighboring countries, high voltage, EP2050 CH electricity production, hard coal 0.07206638306595883 RoW
market for electricity, from neighboring countries, high voltage, EP2050 CH electricity production, natural gas, combined cycle power plant 0.19563488461157266 CH
market for electricity, from neighboring countries, high voltage, EP2050 CH treatment of municipal solid waste, incineration 0.001355966489917591 CH
market for electricity, from neighboring countries, high voltage, EP2050 CH electricity production, photovoltaic 0.09750412452725007 CH
market for electricity, from neighboring countries, high voltage, EP2050 CH electricity production, wind, >3M

In [ ]:
scenarios = [
    {"model": "image", "pathway":"SSP2-Base", "year": 2030},
]

ndb = NewDatabase(
        scenarios = scenarios,        
        source_db="ei 3.8 cutoff",
        source_version="3.8",
        key='tUePmX_S5B8ieZkkM7WUU2CnO8SmShwmAeWK9x2rTFo=',
        external_scenarios=[
            switzerland_2050,
        ]
)

#ndb.update_electricity()
ndb.update_external_scenario()

Once the projections are integrated, you can export the database(s) back to your Brightway2 project, to
a CSV Simapro file, or as a set of sparse amtrices (see main exmaples notebook for more details).

In [121]:
temp_db = ndb.scenarios[0]["database"]

actList = [X for X in temp_db if 'market for electricity' in X['name']                # Iterates trough activities X in database db and adds them to list 'act_list' if they contain 'text1' in 'name',
                        and 'CH' in X['location']                # 'text2' in 'name',
                        and 'high voltage' in X['name']]            # but not 'text3' in 'name',
actList[0]

{'classifications': [('ISIC rev.4 ecoinvent',
   '3510:Electric power generation, transmission and distribution'),
  ('EcoSpold01Categories', 'electricity/supply mix'),
  ('CPC', '17100: Electrical energy')],
 'comment': "The shares of electricity technologies on this market are valid for the year 2019. They have been calculated by the data provider and don't necessarily correspond with the production volumes entered in the undefined datasets of the different electricity producers. The shares have been calculated based on statistics from 2019: Swiss Federal Office of Energy SFOE, Swiss Electricity Statistics 2020; (https://www.bfe.admin.ch/bfe/en/home.html).\nThis is a market activity. Each market represents the consumption mix of a product in a given geography, connecting suppliers with consumers of the same product in the same geographical area. Markets group the producers and also the imports of the product (if relevant) within the same geographical area. They also account for trans

In [101]:
from brightway2 import *
databases

Databases dictionary with 6 object(s):
	TeamPV
	TeamPV2
	TeamPV3
	TeamPV4
	biosphere3
	ei 3.8 cutoff

In [102]:
ndb.write_db_to_brightway(name="TeamPV5")

Write new database(s) to Brightway2.
Prepare database 1.
- check for duplicates...
- check for values format...
- relinking exchanges...
Done!
20899 datasets
649189 exchanges
0 unlinked exchanges
  
Title: Writing activities to SQLite3 database:
  Started: 10/27/2022 15:19:44
  Finished: 10/27/2022 15:20:05
  Total time elapsed: 00:00:21
  CPU %: 93.90
  Memory %: 4.64
Created database: TeamPV5


In [103]:

test=Database("TeamPV5")

In [104]:
len(test)


20899

In [121]:
actList = [X for X in test if 'market for electricity' in X['name']                # Iterates trough activities X in database db and adds them to list 'act_list' if they contain 'text1' in 'name',
                        and 'CH' in X['location']                # 'text2' in 'name',
                        and 'low voltage' in X['name']]            # but not 'text3' in 'name',

In [122]:
actList

['market for electricity, low voltage, EP2050' (kilowatt hour, CH, None),
 'market for electricity, low voltage, renewable energy products' (kilowatt hour, CH, None),
 'market for electricity, low voltage' (kilowatt hour, CH, None)]

In [113]:
EP2050=actList[2]

In [114]:
excList = [exc for exc in EP2050.technosphere()]
excList

[Exchange: 0.026752645518952938 kilowatt hour 'monofacial_tandem_electricity' (kilowatt hour, RER, None) to 'market for electricity, low voltage, EP2050' (kilowatt hour, CH, None)>,
 Exchange: 1.0009965836691253 kilowatt hour 'electricity production, photovoltaic, residential' (kilowatt hour, CH, None) to 'market for electricity, low voltage, EP2050' (kilowatt hour, CH, None)>,
 Exchange: 3.0729701952723535e-09 kilogram 'market for sulfur hexafluoride, liquid' (kilogram, RER, None) to 'market for electricity, low voltage, EP2050' (kilowatt hour, CH, None)>,
 Exchange: 8.982528263103803e-08 kilometer 'distribution network construction, electricity, low voltage' (kilometer, CH, None) to 'market for electricity, low voltage, EP2050' (kilowatt hour, CH, None)>]